In [1]:


import  pandas  as  pd
import  numpy  as  np 
import  time 
import  turicreate  as  tc 
from  sklearn.model_selection  import train_test_split

import  sys

In [2]:
customers = pd.read_csv('../data/recommend_1.csv')
transactions = pd.read_csv('../data/trx_data.csv')

In [3]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [4]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [5]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [6]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [7]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.33 minutes


In [8]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [9]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [10]:
data_dummy = create_data_dummy(data)

In [11]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [13]:
# create a table for input to the modeling  
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [14]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [15]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [16]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [17]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [20]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [21]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23340 users and 300 items.

Data prepared in: 0.189381s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 89525.5

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |     34    | 3.0150375939849625 |  1   |
|    1553    |     37    | 2.9693486590038316 |  2   |
|    1553    |     0     | 2.965746907706946  |  3   |
|    1553    |    132    | 2.890909090909091  |  4   |
|    1553    |     3     | 2.8451612903225807 |  5   |
|    1553    |     27    | 2.7615384615384615 |  6   |
|    1553    |    110    | 2.7515151515151515 |  7   |
|    1553    |    248    | 2.6744186046511627 |  8   |
|    1553    |     32    | 2.6615384615384614 |  9   |
|    1553    |     10    | 2.6491228070175437 |  10  |
|   20400    |     34    | 3.0150375939849625 |  1   |
|   20400    |     37    | 2.9693486590038316 |  2   |
|   20400    |     0     | 2.965746907706946  |  3   |
|   20400    |    132    | 2.890909090909091  |  4   |
|   20400    |     3     | 2.8451612903225807 |  5   |
|   20400 

In [22]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.159845s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 188111

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     2     |  1.0  |  1   |
|    1553    |     17    |  1.0  |  2   |
|    1553    |    209    |  1.0  |  3   |
|    1553    |    148    |  1.0  |  4   |
|    1553    |     62    |  1.0  |  5   |
|    1553    |    231    |  1.0  |  6   |
|    1553    |    216    |  1.0  |  7   |
|    1553    |     3     |  1.0  |  8   |
|    1553    |    199    |  1.0  |  9   |
|    1553    |     35    |  1.0  |  10  |
|   20400    |     2     |  1.0  |  1   |
|   20400    |     17    |  1.0  |  2   |
|   20400    |    209    |  1.0  |  3   |
|   20400    |    148    |  1.0  |  4   |
|   20400    |     62    |  1.0  |  5   |
|   20400    |    231    |  1.0  |  6   |
|   20400    |    216    |  1.0  |  7   |
|   20400    |     3     |  1.0  |  8   |
|   20400    |    199    |  1.0  |  9   |
|   20400    |     35    |  1.0  |  10  |
|   19750    |     2     |  1.0  |

In [23]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23355 users and 300 items.

Data prepared in: 0.159062s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 116306

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7620689655172413 |  1   |
|    1553    |    247    | 0.35400516795865633 |  2   |
|    1553    |    230    | 0.32857142857142796 |  3   |
|    1553    |    125    | 0.25846153846153813 |  4   |
|    1553    |    248    |  0.2554347826086957 |  5   |
|    1553    |    294    | 0.24427480916030503 |  6   |
|    1553    |     72    | 0.23109243697478993 |  7   |
|    1553    |    155    | 0.22962962962962957 |  8   |
|    1553    |    204    | 0.22857142857142845 |  9   |
|    1553    |    276    |      0.2265625      |  10  |
|   20400    |    226    |  0.7620689655172413 |  1   |
|   20400    |    247    | 0.35400516795865633 |  2   |
|   20400    |    230    | 0.32857142857142796 |  3   |
|   20400    |    125    | 0.25846153846153813 |  4   |
|   20400    |    248    |  0.2554347826086957 |

In [42]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23340 users and 300 items.

Data prepared in: 0.275273s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11034004390239716  |  1   |
|    1553    |     35    | 0.08317822217941284  |  2   |
|    1553    |     5     |  0.0628821849822998  |  3   |
|    1553    |     1     | 0.06133013963699341  |  4   |
|    1553    |    124    |  0.056773841381073   |  5   |
|    1553    |     17    | 0.05587776005268097  |  6   |
|    1553    |     33    | 0.05010169744491577  |  7   |
|    1553    |     41    | 0.04913832247257233  |  8   |
|    1553    |     8     | 0.04875198006629944  |  9   |
|    1553    |     21    | 0.04601818323135376  |  10  |
|   20400    |    280    | 0.09283596277236938  |  1   |
|   20400    |    122    | 0.05372977256774902  |  2   |
|   20400    |    284    | 0.048023879528045654 |  3   |
|   20400    |    246    | 0.040438830852508545 |  4   |
|   20400    |    179    | 0.04

Generating candidate set for working with new users.

Finished training in 1.10943s

recommendations finished on 1000/1000 queries. users per second: 19080

In [43]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.225447s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.06126s

recommendations finished on 1000/1000 queries. users per second: 36086.8

In [44]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23355 users and 300 items.

Data prepared in: 0.172641s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.036673s

recommendations finished on 1000/1000 queries. users per second: 41204.8

In [45]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23340 users and 300 items.

Data prepared in: 1.44285s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |     34    | 2.9905111171249157 |  1   |
|    1553    |     37    | 2.9693486590038334 |  2   |
|    1553    |     0     | 2.9607981575686644 |  3   |
|    1553    |    132    | 2.8909090909090907 |  4   |
|    1553    |     3     | 2.8451612903225794 |  5   |
|    1553    |     27    | 2.761538461538463  |  6   |
|    1553    |    110    | 2.744640411391403  |  7   |
|    1553    |    248    | 2.674418604651162  |  8   |
|    1553    |     32    | 2.661538461538462  |  9   |
|    1553    |     10    | 2.648196312419154  |  10  |
|   20400    |     34    | 3.015037593984962  |  1   |
|   20400    |     37    | 2.967069436992267  |  2   |
|   20400    |     0     | 2.965746907706947  |  3   |
|   20400    |    132    | 2.879421130093661  |  4   |
|   20400    |     3     | 2.8436961485493555 |  5   |
|   20400 

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.12107s

recommendations finished on 1000/1000 queries. users per second: 26773

In [46]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.15782s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.05244s

recommendations finished on 1000/1000 queries. users per second: 38321.5

In [47]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23355 users and 300 items.

Data prepared in: 0.502331s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.048792s

recommendations finished on 1000/1000 queries. users per second: 23174.4

ToolkitError: Target column given as 'scaled_purchase_freq', but this is not present in the provided data.